In [1]:
import os

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import torch
import math 
import copy
import random
import argparse
import torch.optim as optim



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
def load_data(dataset_name, K):
    if dataset_name == "statlog":
        ##statlog
        __context_size__ = 8
        __arm_size__ = 7
        fr = open("shuttle.trn",'r+')
        CONTEXT = []
        REWARD = []
        SYM = {}
        aa = 0
        K = 0
        for line in fr:
            K+=1
            aaa = line.split("\n")
            aaa = aaa[0].split(" ")
            temp = np.int64(aaa[9])-1
            context = np.double(aaa[1:9])
            context = context/np.linalg.norm(context)
            ttt = context
            CONTEXT.append(ttt)
            reward = np.zeros(__arm_size__)
            reward[temp] = 1
            REWARD.append(reward)

    elif dataset_name == "magic":
        ####magic gamma

        __context_size__ = 10
        __arm_size__ = 2
        fr = open("letter.data", 'r+')

        K = 0
        CONTEXT = []
        REWARD = []

        fr.close()
        fr = open("magic04.data", 'r+')
        for line in fr:

            context = []
            aaa = line.split(",")
            context = np.double(aaa[0:__context_size__])
            context = context / np.linalg.norm(context)
            K += 1
            CONTEXT.append(np.array(context))
            __context_size__ = len(context)
            reward = np.zeros(__arm_size__)
            if aaa[10] == 'g\n':
                reward[0] = 1
            else:
                reward[1] = 1
            REWARD.append(reward)

        fr.close()
    elif dataset_name == "covertype":
        ##For covertypr
        __context_size__ = 14
        __arm_size__ = 7
        fr = open("covtype.data", 'r+')
        CONTEXT = []
        REWARD = []
        SYM = {}
        aa = 0
        K = 0
        for line in fr:
            K += 1
            aaa = line.split(",")
            __context_size__ = len(aaa) - 1
            temp = int(aaa[__context_size__]) - 1
            context = np.double(aaa[0:__context_size__])
            context = context / np.linalg.norm(context)
            ttt = context
            CONTEXT.append(ttt)
            reward = np.zeros(__arm_size__)
            reward[temp] = 1
            REWARD.append(reward)
    else:
        assert('dataset not avaiable')

    TEMP_CONTEXT = []
    TEMP_REWARD = []
    sli = np.random.permutation(K)
    for i in range(K):
        TEMP_CONTEXT.append(CONTEXT[sli[i]])
        TEMP_REWARD.append(REWARD[sli[i]])
    CONTEXT = TEMP_CONTEXT
    REWARD = TEMP_REWARD

    return __context_size__, __arm_size__, CONTEXT, REWARD, sli

In [3]:
# define the hyperparameters
input_size = 15
hidden_size = 50
output_size = 15
num_layers = 10
arm_size = 7
beta = 0.1
lambd = 1
H = 100

In [4]:
# initialize the parameters

theta = np.random.randn(output_size, 1) / np.sqrt(output_size)
theta = torch.tensor(theta, dtype = torch.float32)

LAMBDA = lambd * torch.eye(output_size, dtype=torch.float32)
bb = torch.zeros(LAMBDA.size()[0], dtype=torch.float).view(-1, 1)

In [5]:
# create the model 
import modeldefine
import torch.nn as nn
model = modeldefine.Model(input_size, hidden_size, output_size, num_layers).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.01)

class CustomMSELoss(nn.Module):
    def __init__(self, theta):
        super(CustomMSELoss, self).__init__()
        self.theta = theta
        self.mse_loss = nn.MSELoss()

    def forward(self, output, target):
        prediction = torch.sum(output * self.theta, dim=1)  # 按元素乘法并求和
        return self.mse_loss(prediction, target)


tensor([15, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 15], dtype=torch.int32)


In [6]:
def TRANS(c, a, arm_size):
    #### transfer an array context + action to new context with dimension 2*(__context__ + __armsize__)
    dim = len(c)
    action = np.zeros(arm_size)
    action[a] = 1
    c_final = np.append(c, action)
    c_final = torch.from_numpy(c_final).to(device)
    c_final = c_final.view((len(c_final), 1))
    c_final = c_final.repeat(2, 1)
    c_final = torch.tensor(c_final, dtype=torch.float32).view(1,-1)
    return c_final

def UCB(A, phi):
    #### ucb term
    phi = phi.view(-1,1)
    try:
        tmp, LU = torch.linalg.solve(phi,A)
    except:
        A = A.detach().numpy()
        phi2 = phi.detach().numpy()
        tmp = torch.Tensor(np.linalg.solve(A, phi2))

    return torch.sqrt(torch.matmul(torch.transpose(phi,1,0), tmp))

def get_action(model, context_info, random_indicator):
    ucb = [] 
    bphi = []
    for i in range(arm_size):
        temp = TRANS(context_info, i, arm_size).to(device)
        bphi.append(temp.cpu())
        feature = model(temp).cpu()

        first_item = torch.mm(feature, theta)
        ucb_item = beta * UCB(LAMBDA, feature)
        ucb.append(first_item + ucb_item)
    
    if random_indicator == 0:
        return ucb.index(max(ucb)), bphi[ucb.index(max(ucb))] 
    else:
        num =  np.random.randint(0, arm_size)
        return num, bphi[num]
    
def update_lambda_bb(lambd, bb, context_info, action,reward):
    temp = TRANS(context_info, action, arm_size).to(device)
    feature = model(temp).cpu()
    templambda = lambd + torch.mm(feature.t(), feature)
    tempbb = bb + reward * feature.t()
    return templambda, tempbb

def update_theta(lambd, bb):
    return torch.linalg.solve(lambd, bb)

In [13]:
THETA_action = []
CONTEXT_action = []
REWARD_action = []
context_size, arm_size, CONTEXT_list, REWARD, sli = load_data("statlog", 15000)

for t in range(0,500):
    context = CONTEXT_list[t]
    if t <= 21:
        action, c_act = get_action(model, context, 1)
    else: 
        action, c_act = get_action(model, context, 0) 

    # get the reward
    reward = REWARD[t][action] 

    # construct the training set for neural network
    if t % H == 0:
        CONTEXT_action = []
        REWARD_action = []
        CONTEXT_action = c_act
        REWARD_action = torch.tensor([reward],device= device, dtype=torch.float32)
    else:
        CONTEXT_action = torch.cat((CONTEXT_action, c_act), 0)
        REWARD_action = torch.cat((REWARD_action, torch.tensor([reward],device= device, dtype=torch.float32)), 0)

    # update lambda, bb and theta
    LAMBDA, bb = update_lambda_bb(LAMBDA,bb,context, action, reward)
    theta = update_theta(LAMBDA, bb)

    if t % H == 0:
        THETA_action = []
        THETA_action = theta.view(1,-1)
    else: 
        THETA_action = torch.cat((THETA_action, theta.view(1,-1)), 0)
    
    # update the model
    loss_function = CustomMSELoss(THETA_action.detach().to(device))

    epochs = 10
    for epoch in range(epochs):
        # 前向传播
        outputs = model(CONTEXT_action.to(device))
        loss = loss_function(outputs, REWARD_action)
        
        # 打印损失
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}")
        
        # 反向传播和优化
        optimizer.zero_grad()  # 重要：每次迭代前清零梯度
        loss.backward()
        optimizer.step()
    print('------')
    

C:\Users\杜骏晔\AppData\Local\Temp\ipykernel_37824\3790411606.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  c_final = torch.tensor(c_final, dtype=torch.float32).view(1,-1)


Epoch [1/10], Loss: 0.0
Epoch [2/10], Loss: 0.0
Epoch [3/10], Loss: 0.0
Epoch [4/10], Loss: 0.0
Epoch [5/10], Loss: 0.0
Epoch [6/10], Loss: 0.0
Epoch [7/10], Loss: 0.0
Epoch [8/10], Loss: 0.0
Epoch [9/10], Loss: 0.0
Epoch [10/10], Loss: 0.0
------
Epoch [1/10], Loss: 0.33002930879592896
Epoch [2/10], Loss: 0.24862192571163177
Epoch [3/10], Loss: 0.17075175046920776
Epoch [4/10], Loss: 0.10294345766305923
Epoch [5/10], Loss: 0.05033940449357033
Epoch [6/10], Loss: 0.01930590346455574
Epoch [7/10], Loss: 0.002351516392081976
Epoch [8/10], Loss: 0.002036656253039837
Epoch [9/10], Loss: 0.014183223247528076
Epoch [10/10], Loss: 0.025454942137002945
------
Epoch [1/10], Loss: 0.0188310407102108
Epoch [2/10], Loss: 0.01663810759782791
Epoch [3/10], Loss: 0.011966927908360958
Epoch [4/10], Loss: 0.0074290931224823
Epoch [5/10], Loss: 0.0035942704416811466
Epoch [6/10], Loss: 0.0012157741002738476
Epoch [7/10], Loss: 0.0002230893005616963
Epoch [8/10], Loss: 0.00013650428445544094
Epoch [9/10]

In [11]:
class CustomMSELoss(torch.nn.Module):
    def __init__(self, theta):
        super(CustomMSELoss, self).__init__()
        self.theta = theta
        self.mse_loss = torch.nn.MSELoss()

    def forward(self, output, target):
        prediction = torch.sum(output * self.theta, dim=1)  # 按元素乘法并求和
        return self.mse_loss(prediction, target)
    
loss_function = CustomMSELoss(THETA_action.to(device))
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 仅优化神经网络的参数

# ... (其他代码，如模型、损失函数和优化器的定义等)

epochs = 10
for epoch in range(epochs):
    # 前向传播
    outputs = model(CONTEXT_action.to(device))
    loss = loss_function(outputs, REWARD_action)
    
    # 打印损失
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}")

    # 反向传播和优化
    optimizer.zero_grad()  # 重要：每次迭代前清零梯度
    loss.backward()
    optimizer.step()

Epoch [1/10], Loss: 0.0


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [ ]:
REWARD_action

tensor([0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
        1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
        1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 0., 1., 0., 0.], device='cuda:0')

In [ ]:

import torch.nn as nn
model = modeldefine.Model(input_size, hidden_size, output_size, num_layers).to(device)
# 定义自定义的MSE损失
class CustomMSELoss(nn.Module):
    def __init__(self, theta):
        super(CustomMSELoss, self).__init__()
        self.theta = theta
        self.mse_loss = nn.MSELoss()

    def forward(self, output, target):
        prediction = torch.sum(output * self.theta, dim=1)  # 按元素乘法并求和
        return self.mse_loss(prediction, target)

# 示例用法


# 随机数据和theta
#inputs = torch.randn(batch_size, input_dim).to(device)
#theta_values = torch.randn(batch_size, output_dim).to(device)
#targets = torch.randn(batch_size).to(device)

inputs = CONTEXT_action.to(device)
targets = REWARD_action.to(device)

# 创建模型、损失函数和优化器
theta_values = THETA_action.detach().to(device)
loss_function = CustomMSELoss(theta_values)
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 仅优化神经网络的参数

# ... (其他代码，如模型、损失函数和优化器的定义等)

epochs = 10
for epoch in range(epochs):
    # 前向传播
    outputs = model(inputs)
    loss = loss_function(outputs, targets)
    
    # 打印损失
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}")

    # 反向传播和优化
    optimizer.zero_grad()  # 重要：每次迭代前清零梯度
    loss.backward()
    optimizer.step()

tensor([15, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 15], dtype=torch.int32)
Epoch [1/10], Loss: 0.9890469908714294
Epoch [2/10], Loss: 0.280592143535614
Epoch [3/10], Loss: 0.16444365680217743
Epoch [4/10], Loss: 0.3059827983379364
Epoch [5/10], Loss: 0.31880253553390503
Epoch [6/10], Loss: 0.24453036487102509
Epoch [7/10], Loss: 0.1797829419374466
Epoch [8/10], Loss: 0.16122767329216003
Epoch [9/10], Loss: 0.17066682875156403
Epoch [10/10], Loss: 0.1804012656211853
